In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import requests
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Data Preprocessing

In [ ]:
data = pd.read_csv("databaru.csv")

In [ ]:
data

In [ ]:
data['Weather'] = data['Weather'].str.strip()

In [ ]:
weather_mapping = {'Sunny': 0, 'Rain': 1}
data['Weather'] = data['Weather'].map(weather_mapping)

In [ ]:
print(data['Weather'])

In [ ]:
print(data['City_1'].unique())

In [ ]:
label_encoder = LabelEncoder()
data['City_1'] = label_encoder.fit_transform(data['City_1'])

In [ ]:
X = data[['Weather', 'City_1']].values
y = data['Place_Name'].values

# Weather & User Data Retrieval

In [ ]:
city_coordinates = {
    "Jakarta Pusat": {"lat": -6.186486, "lon": 106.834091},
    "Jakarta Barat": {"lat": -6.135200, "lon": 106.813301},
    "Jakarta Timur": {"lat": -6.230702, "lon": 106.882744},
    "Jakarta Selatan": {"lat": -6.300641, "lon": 106.814095}
}

In [ ]:
user_city = input("Enter your city: ")
if user_city not in city_coordinates:
    print("Invalid city input!")
    exit()

In [ ]:
user_lat = city_coordinates[user_city]["lat"]
user_lon = city_coordinates[user_city]["lon"]

In [ ]:
print(user_lat)
print(user_lon)

In [ ]:
api_key = "6efe5d86568540c60dfc5ba4a75a7bbe"
api_url = f"http://api.openweathermap.org/data/2.5/weather?lat={user_lat}&lon={user_lon}&appid={api_key}"
response = requests.get(api_url)
print(response)

In [ ]:
weather_description = response.json()["weather"][0]["main"].lower()

In [ ]:
weather_mapping = {'clear': 'sunny', 'clouds': 'sunny', 'haze': 'sunny', 'rain': 'rain', 'drizzle': 'rain', 'thunderstorm': 'rain', 'squall': 'rain'}
user_weather = weather_mapping.get(weather_description, 'unknown')

In [ ]:
print(f"User City: {user_city}")
print(f"Weather Description: {weather_description}")
print(f"Mapped Weather: {user_weather}")

# Model Training

In [ ]:
weather_categories = ['sunny', 'rain']

In [ ]:
user_weather_encoded = [int(user_weather.lower() == category) for category in weather_categories]

In [ ]:
print(user_weather_encoded)

In [ ]:
user_weather_encoded = np.array([[user_weather_encoded]])

In [ ]:
#filter citynya

In [ ]:
city_filter = data['City_1'] == label_encoder.transform([user_city])[0]

In [ ]:
filtered_X = X[city_filter]
filtered_y = y[city_filter]

In [ ]:
filtered_label_encoder = LabelEncoder()

In [ ]:
filtered_y_encoded = filtered_label_encoder.fit_transform(filtered_y)

In [ ]:
print(filtered_y_encoded)

In [ ]:
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(filtered_X, filtered_y_encoded, test_size=0.2, random_state=42)

In [ ]:
num_classes = len(np.unique(y_train_encoded))

In [ ]:
feature_dim = X_train.shape[1]

In [ ]:
unique_labels = np.unique(y_train_encoded)

In [ ]:
valid_indices = np.where(unique_labels < num_classes)[0]

In [ ]:
valid_labels = unique_labels[valid_indices]

In [ ]:
valid_indices_mask = np.isin(y_train_encoded, valid_labels)

In [ ]:
X_train_valid = X_train[valid_indices_mask]
y_train_valid = y_train_encoded[valid_indices_mask]

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(feature_dim,)),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train_valid, y_train_valid, epochs=10)

# Recommendation

In [ ]:
user_weather_encoded = np.reshape(user_weather_encoded, (1, -1))

In [ ]:
predicted_probabilities = model.predict(user_weather_encoded)

In [ ]:
top_5_indices = np.argsort(predicted_probabilities[0])[-5:][::-1]

In [ ]:
recommended_attractions = filtered_label_encoder.inverse_transform(top_5_indices)
print(f"\nRecommended attractions for {user_city} with {weather_description}:")
for attraction in recommended_attractions:
    print(attraction)

# Sa

In [ ]:
import tensorflow as tf

# Load the model from the .h5 file
model = tf.keras.models.load_model("C:/Users/DAffa/Documents/model.h5")

# Convert the model to JSON format
model_json = model.to_json()

# Save the model in JSON format
with open("C:/Users/DAffa/Documents/model.json", "w") as json_file:
    json_file.write(model_json)

print("Model saved in JSON format.")
